### 1. 基本概念
1. 定义
- LangChain中的嵌入模型接口
- 基于Ollama的文本向量化工具
- 支持多种底层嵌入模型
2. 主要功能

In [2]:
from langchain_community.embeddings import OllamaEmbeddings

# 基本初始化
embeddings = OllamaEmbeddings(
    model="bge-m3",  # 使用的模型
    base_url="http://localhost:11434"  # Ollama服务地址
)

#作用：
# base_url 告诉程序在哪里找到运行的 Ollama 服务
# 默认情况下，Ollama 服务在本地运行，监听 11434 端口
# 通过这个 URL，程序可以向 Ollama 服务发送请求，获取文本嵌入
# 参数组成：
# http://localhost：表示服务在本地运行
# 11434：是 Ollama 的默认端口号
# 这个端口号是在安装 Ollama 时自动配置的

# 生成文本嵌入
text = "这是一段示例文本"
text1 = "这是一段示例文本1"
text2 = "这是一段示例文本2"
vector = embeddings.embed_query(text)  # 生成单个文本的向量
vectors = embeddings.embed_documents([text1, text2])  # 批量生成向量

### 2. 工作原理
1. 向量化过程
   - 文本预处理
   - 模型编码
   - 向量生成
   - 维度规范化
2. 技术细节

In [ ]:
# 自定义配置示例
embeddings = OllamaEmbeddings(
    model="bge-m3",
    num_ctx=4096,         # 上下文窗口大小
    num_thread=4,         # 使用的线程数
    show_progress=True,   # 显示进度
    timeout=30            # 超时设置
)

### 3. 使用场景
1. 文本相似度计算

In [ ]:
# 计算两段文本的相似度
from numpy import dot
from numpy.linalg import norm

def compute_similarity(text1, text2):
    vec1 = embeddings.embed_query(text1)
    vec2 = embeddings.embed_query(text2)
    return dot(vec1, vec2)/(norm(vec1)*norm(vec2))

2. 向量数据库集成

In [ ]:
from langchain_community.vectorstores import Chroma

# 创建向量存储
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

  3.语义搜索

In [ ]:
# 实现语义搜索
def semantic_search(query, documents, top_k=3):
    query_vector = embeddings.embed_query(query)
    doc_vectors = embeddings.embed_documents(documents)
    
    similarities = [
        dot(query_vector, doc_vec)/(norm(query_vector)*norm(doc_vec))
        for doc_vec in doc_vectors
    ]
    
    return sorted(
        zip(documents, similarities),
        key=lambda x: x[1],
        reverse=True
    )[:top_k]

### 4. 高级用法
1. 批量处理优化

In [ ]:
# 批量处理文档
def process_documents_in_batches(documents, batch_size=32):
    results = []
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        vectors = embeddings.embed_documents(batch)
        results.extend(vectors)
    return results

2.错误处理

In [ ]:
# 带重试的嵌入生成
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def embed_with_retry(text):
    try:
        return embeddings.embed_query(text)
    except Exception as e:
        print(f"Embedding error: {e}")
        raise

3.异步处理

In [ ]:
# 异步嵌入生成
async def async_embed_documents(documents):
    tasks = []
    for doc in documents:
        task = embeddings.aembed_query(doc)
        tasks.append(task)
    return await asyncio.gather(*tasks)

### 5. 性能优化建议
1. 模型选择


In [ ]:
# 不同场景的模型选择
embeddings_fast = OllamaEmbeddings(model="bge-small-en")  # 轻量快速
embeddings_accurate = OllamaEmbeddings(model="bge-m3")    # 高精度

2. 缓存机制


In [ ]:
# 实现简单的缓存
from functools import lru_cache

class CachedEmbeddings:
    def __init__(self, embedding_model):
        self.model = embedding_model
    
    @lru_cache(maxsize=1000)
    def embed_query(self, text):
        return self.model.embed_query(text)

3. 资源管理

In [ ]:
# 资源使用监控
import psutil
import GPUtil

def monitor_resources():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    if GPUtil.getGPUs():
        gpu = GPUtil.getGPUs()[0]
        gpu_load = gpu.load
        gpu_memory = gpu.memoryUtil
        return {
            "cpu": cpu_percent,
            "memory": memory_percent,
            "gpu_load": gpu_load,
            "gpu_memory": gpu_memory
        }
    return {
        "cpu": cpu_percent,
        "memory": memory_percent
    }

### 6. 实际应用示例
1. 文档检索系统


In [ ]:
class DocumentRetrieval:
    def __init__(self, documents):
        self.embeddings = OllamaEmbeddings(model="bge-m3")
        self.vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings
        )
    
    def search(self, query, k=3):
        return self.vectorstore.similarity_search(query, k=k)

2. 文本聚类


In [ ]:
from sklearn.cluster import KMeans

def cluster_texts(texts, n_clusters=5):
    # 生成嵌入向量
    vectors = embeddings.embed_documents(texts)
    
    # 执行聚类
    kmeans = KMeans(n_clusters=n_clusters)
    clusters = kmeans.fit_predict(vectors)
    
    # 整理结果
    results = {}
    for text, cluster in zip(texts, clusters):
        if cluster not in results:
            results[cluster] = []
        results[cluster].append(text)
    
    return results


OllamaEmbeddings是一个强大的文本向量化工具，通过合理使用可以支持多种高级AI应用的开发。它的性能和易用性使其成为构建语义搜索、文档检索等应用的理想选择。